Deep Learning Model

In [ ]:
import keras.layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Dropout
from tensorflow.keras import losses, regularizers, metrics

In [ ]:
regulization_value = 0.000001


In [ ]:
# Pass Extracted Features
initial_input = Input(shape=(None, 42), name="Input")

dense1 = Dense(24, name="Dense_1", activation="tanh")(initial_input)

# GRU - Voice Activity Detection
gru1 = GRU(24, name="GRU_1", activation="tanh",
           recurrent_activation="sigmoid", return_sequences=True,
           kernel_regularizer=regularizers.l2(regulization_value),
           recurrent_regularizer=regularizers.l2(regulization_value))(dense1)

# Voice Activity Detection - Output
dense2 = Dense(1, name="Dense_2", activation="sigmoid")(gru1)

# Noise Input
concat_input1 = keras.layers.concatenate([dense1, gru1, initial_input])

# GRU with Noise Input
gru2 = GRU(48, name="GRU_2", activation="relu",
           recurrent_activation="sigmoid", return_sequences=True,
           kernel_regularizer=regularizers.l2(regulization_value),
           recurrent_regularizer=regularizers.l2(regulization_value))(concat_input1)

# Denoise Input
concat_input2 = keras.layers.concatenate([gru1, gru2, initial_input])

# GRU with Denoise Input
gru3 = GRU(96, name="GRU_3", activation="tanh",
           recurrent_activation="sigmoid", return_sequences=True,
           kernel_regularizer=regularizers.l2(regulization_value),
           recurrent_regularizer=regularizers.l2(regulization_value))(concat_input2)

# Denoise Output
final_output = Dense(22, name="Final_Output", activation="sigmoid")(gru3)

# Model
model = Model(inputs=initial_input, outputs=[final_output, dense1])

# Get Model Summary
model.summary()

# model.compile(optimizer="Adam", loss="mse", metrics=[metrics.mse])